I am a big fan of korean dramas. I've been watching thm since I was in junior high. With the huge variety of korean dramas it's hard for me to know which are worth my time to watch. I'm hoping that I can use ML and NLP to provide accurate predictions for kdrama fans.

The first issue I have is figuring out how to get a large database of kdramas. I do not have a ton of experience with webscraping so using some of the kdrama websites is a little hard. I decided to opt for getting a list of kdramas from wikipedia first and then cross it with imdb database to extract information there.

In [1]:
import requests
import pandas as pd

kdramaWikiPage = requests.get("https://en.wikipedia.org/wiki/List_of_South_Korean_dramas")

from bs4 import BeautifulSoup
soup = BeautifulSoup(kdramaWikiPage.text, 'lxml')
#print(soup.prettify())

In [2]:
koreanDramaNames = []
KoreanDramaYears = []

for dramaListed in soup.find_all('li'):
    subDramaListed = dramaListed.find_all('ul')
    if len(subDramaListed) > 0:
        for kDramaName in subDramaListed[0].find_all('i'):
            koreanDramaNames.append(kDramaName.text.strip())
            nameAndyear = subDramaListed[0].text
            startIndex = nameAndyear.index("(") + 1
            KoreanDramaYears.append(nameAndyear[startIndex:-1])
    else:
        allKdramas = dramaListed.find_all('i')
        if len(allKdramas) > 0:
            kDramaName = allKdramas[0].text
            if kDramaName.strip() != "":
                dramaListedText = dramaListed.text
                koreanDramaNames.append(kDramaName.strip())
                startIndex = dramaListedText.index("(") + 1
                KoreanDramaYears.append(dramaListedText[startIndex:-1])

wikiKdramas = {'title': koreanDramaNames, 'year':KoreanDramaYears}
wikiKdramasDf = pd.DataFrame(wikiKdramas)

wikiKdramasDf.head()

,title,year
0,100 Days My Prince,2018
1,12 Signs of Love,2012
2,12 Years Promise,2014
3,18 Again,2020
4,365: Repeat the Year,2020


In [3]:
wikiKdramasDf.shape

(1574, 2)

The years via the kdrama wiki page have different formats. I need a uniform start and end date. The reason for this is because I don't know what year format imdb uses. This will be extremely important for later.

In [4]:
wikiKdramasDf = wikiKdramasDf.loc[wikiKdramasDf['year'].str.len() < 13]
wikiKdramasDf['year'].unique()

array(['2018', '2012', '2014', '2020', '2014–15', '2011', '2005',
       '2016–17', '2013', '2006–07', '2016', '2015', '2007', '2017–18',
       '2013–14', '2020–21', '2010', '2022', '2019', '2018–19', '2011–12',
       '2023', '2021', '2008', '2002', '2003–04', '2000', '2015–16',
       '2003', '2010–11', '2006', '2004', '2017', '1995', '2009–10',
       '2005–06', '2004–06', '2001', '2019–20', '2009', '2021–22',
       '2012–13', 'cancelled', '2007–08', '1981', '1999', '2008–09',
       '2016–20', '2004–05', '2001–02', '1991–92', '1996–97', '1983',
       'TBA', '1999–2000', '1993', '1975–78', '2019–present', '2019–21',
       '2017–19', '2002–04', '2002–03', '1997', '2000–01', '2000–06',
       '1996', '2010–19', '1994–95', '2003–07', '2011–13', '2017–present',
       '2000–02', '1996–98', '1999–2009', '1998–2000', '1994',
       '2007–2019', '2019–2020', '1998', '1997–98'], dtype=object)

In [5]:
wikiKdramasDf = wikiKdramasDf[~wikiKdramasDf['year'].isin(['cancelled', 'TBA'])]
wikiKdramasDf.shape

(1522, 2)

In [6]:
def getEndDate(year):
    splitStartAndEnd = year.split("–")
    start = splitStartAndEnd[0]
    if len(splitStartAndEnd) > 1:
        if splitStartAndEnd[1] == "present":
            end = 2023
        else:
            end = splitStartAndEnd[1] if len(splitStartAndEnd[1]) > 2 else start[:2]+splitStartAndEnd[1]
    else:
        end = start
    return int(end)

wikiKdramasDf['endYear'] = wikiKdramasDf['year'].map(lambda year: getEndDate(year))
wikiKdramasDf['year'] = wikiKdramasDf['year'].map(lambda year: int(year.split("–")[0]))

wikiKdramasDf

,title,year,endYear
0,100 Days My Prince,2018,2018
1,12 Signs of Love,2012,2012
2,12 Years Promise,2014,2014
3,18 Again,2020,2020
4,365: Repeat the Year,2020,2020
...,...,...,...
1569,Your Scene,2007,2007
1570,Youth,2021,2021
1571,Youth of May,2021,2021
1572,Yumi's Cells,2021,2022


In [7]:
wikiKdramasDf['year'].unique()

array([2018, 2012, 2014, 2020, 2011, 2005, 2016, 2013, 2006, 2015, 2007,
       2017, 2010, 2022, 2019, 2023, 2021, 2008, 2002, 2003, 2000, 2004,
       1995, 2009, 2001, 1981, 1999, 1991, 1996, 1983, 1993, 1975, 1997,
       1994, 1998])

In [8]:
wikiKdramasDf.head()

,title,year,endYear
0,100 Days My Prince,2018,2018
1,12 Signs of Love,2012,2012
2,12 Years Promise,2014,2014
3,18 Again,2020,2020
4,365: Repeat the Year,2020,2020


In [9]:
from imdb import Cinemagoer
ia = Cinemagoer()

for idx, d in wikiKdramasDf.iterrows():
    title = d['title']
    year = d['year']
    found = []
    
    try:
        dramas = ia.search_movie_advanced(title)
        for dr in dramas:
            #Match by Year
            try:
                if (dr['title'] == title and dr['year'] == year):
                    found.append(dr)
            except:
                print(title, 'year not found')
        
        # If found > 1 drama with same name
        if len(found) == 1:
            wikiKdramasDf.loc[wikiKdramasDf['title'] == d['title'], 'movie_id'] = found[0].movieID
        elif len(found) > 1:
            wikiKdramasDf.loc[wikiKdramasDf['title'] == d['title'], 'movie_id'] = [x.movieID for x in found]

        print("Drama: ", title, "success")
    except:
        print("Drama: ", title, "error")

Drama:  100 Days My Prince success
Drama:  12 Signs of Love success
Drama:  12 Years Promise success
Drama:  18 Again success
Drama:  365: Repeat the Year success
Drama:  4 Legendary Witches success
49 Days year not found
Drama:  49 Days success
Drama:  5th Republic success
Drama:  7 First Kisses success
Drama:  7th Grade Civil Servant success
Drama:  90 Days, Time to Love success
Drama:  A Beautiful Mind success
Drama:  A Bird That Doesn't Sing success
Drama:  A Daughter Just Like You success
Drama:  A Gentleman's Dignity success
Drama:  A Girl Who Sees Smells success
Drama:  A Happy  Woman success
Drama:  A Hundred Year Legacy success
Drama:  A Korean Odyssey success
Drama:  A Little Love Never Hurts success
Drama:  A Love So Beautiful success
Drama:  A Love to Kill success
Drama:  A Man Called God success
Drama:  A Model Family success
Drama:  A New Leaf success
Drama:  A Piece of Your Mind success
Drama:  A Place in the Sun success
Drama:  A Pledge to God success
Drama:  A Poem a D

2022-10-31 20:41:58,694 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


Drama:  Be Positive error
Drama:  Be Strong, Geum-soon! success
Drama:  Beating Again success
Drama:  Beating Heart success
Drama:  Beautiful Days success
Drama:  Beautiful Gong Shim success
Drama:  Beautiful Love, Wonderful Life success
Drama:  Beautiful World success
Drama:  Because This Is My First Life error
Drama:  Because This Is My First Twenty success
Drama:  Becky's Back success
Drama:  Becoming a Billionaire success
Drama:  Becoming Witch success
Drama:  Beethoven Virus success
Drama:  Behind the White Tower success
Drama:  Bel Ami success
Drama:  Best Chicken success


2022-10-31 20:42:11,631 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


Drama:  Beyond Evil error
Beyond the Clouds year not found
Drama:  Beyond the Clouds success
Drama:  Big success
Drama:  Big Issue error
Drama:  Big Man success
Drama:  Big Mouth success
Drama:  Big Thing success
Drama:  Billie Jean, Look at Me success
Drama:  Birdie Buddy success
Drama:  Birth of a Beauty success
Drama:  Birthcare Center success
Drama:  Birthday Letter success
Drama:  Bitter Sweet Life success
Drama:  Bizarre Bunch success
Drama:  Black success
Drama:  Black Dog: Being A Teacher success
Black Knight year not found
Black Knight year not found
Black Knight year not found
Drama:  Black Knight success
Drama:  Black Knight: The Man Who Guards Me success
Drama:  Blade Man success
Drama:  Blessing of the Sea success
Drama:  Blind success
Drama:  Blood success
Drama:  Blooded Palace: The War of Flowers success
Drama:  Bloody Heart success
Drama:  Bloody Romance success
Drama:  Blossom Sisters success
Drama:  Blow Breeze success
Drama:  Blue Birthday success
Drama:  Bodyguard 

...

Drama:  The President error
Drama:  The Princess' Man success
Drama:  The Producers success
Drama:  The Promise success
Drama:  The Queen of Office success
Drama:  The Queen's Classroom success
Drama:  The Rebel success
Drama:  The Red Sleeve success
Drama:  The Reputable Family success
Drama:  The Return of Hwang Geum-bok success
Drama:  The Return of Iljimae success
Drama:  The Rich Son success
The Road Home year not found
Drama:  The Road Home success
Drama:  The Road: The Tragedy of One success
Drama:  The Royal Gambler success
Drama:  The Running Mates: Human Rights success
Drama:  The Scandal success
Drama:  The Scholar Who Walks the Night success
The School Nurse Files year not found
Drama:  The School Nurse Files success
Drama:  The Second Husband success
Drama:  The Secret House success
Drama:  The Secret Life of My Secretary success
Drama:  The Secret Lovers success
Drama:  The Secret of Birth success
Drama:  The Secret of My Love success
Drama:  The Silent Sea error
Drama:  

2022-10-31 20:58:18,524 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


Drama:  Thirty-Nine error
Drama:  This Is My Love success
Drama:  Thousand Years of Love success
Drama:  Three Bold Siblings success
Drama:  Three Brothers success
Drama:  Queen of the Ring success
Drama:  Romance Full of Life success
Drama:  The Universe's Star success
Drama:  Three Days success
Drama:  Three Friends success
Drama:  Three Sisters success
Drama:  Thrice Married Woman success
Drama:  Through the Darkness success
Drama:  Time Between Dog and Wolf success
Drama:  Times success
Drama:  To My Beloved success
Drama:  To the Beautiful You success
Drama:  To. Jenny success
Drama:  Today's Webtoon success
Drama:  Toji, the Land success
Drama:  Tokyo Sun Shower success
Drama:  Tomorrow success
Tomorrow Boy year not found
Drama:  Tomorrow Boy success
Drama:  Tomorrow, with You success
Drama:  Top Management success
Drama:  Top Star U-back success
Drama:  Touch success
Drama:  Touch Your Heart success
Drama:  Tracer error
Drama:  Train success
Drama:  Trap success
Drama:  Tree of 

2022-10-31 20:58:50,095 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


Drama:  Triple error
Drama:  Trolley success
Drama:  True Beauty success
Drama:  Tunnel success
Drama:  Twelve Nights success
Drama:  Twenty-Five Twenty-One success
Twinkle Twinkle year not found
Drama:  Twinkle Twinkle success
Drama:  Two Cops success
Drama:  Two Mothers success
Drama:  Two Outs in the Ninth Inning success
Drama:  Two Weeks success
Drama:  Two Wives success
Drama:  Two Women's Room success
Drama:  Ugly Alert success
Drama:  Ugly Miss Young-ae success
Drama:  Unasked Family success
Drama:  Uncle success
Drama:  Uncontrollably Fond success
Drama:  Under the Queen's Umbrella success
Drama:  Undercover error
Drama:  Understanding of Love success
Drama:  Unframed success
Drama:  Unkind Ladies success
Drama:  Unknown Woman success
Drama:  Unlock the Boss success
Drama:  Unstoppable Marriage error
Drama:  Until the Azalea Blooms success
Drama:  Untouchable error
Vagabond year not found
Drama:  Vagabond success
Drama:  Vampire Idol success
Drama:  Vampire Prosecutor success
D

In [10]:
wikiKdramasDf[wikiKdramasDf['movie_id'].isnull()]

,title,year,endYear,movie_id
1,12 Signs of Love,2012,2012,NaN
5,4 Legendary Witches,2014,2015,NaN
7,5th Republic,2005,2005,NaN
8,7 First Kisses,2016,2017,NaN
10,"90 Days, Time to Love",2006,2007,NaN
...,...,...,...,...
1559,You Don't Know Women,2017,2017,NaN
1562,You're Beautiful,2009,2009,NaN
1563,You're Only Mine,2014,2014,NaN
1569,Your Scene,2007,2007,NaN


In [11]:
wikiKdramasDf[~wikiKdramasDf['movie_id'].isnull()].shape

(811, 4)

One of the issues I have found with IMDB's api is that it's not accurate to what is current in their production. What does this mean? It means if I search the drama "4 Legendary Witches" on their website  can cleary see a movie_id and a drama tht matches the date I collected. The code below reflects such issues.

Other dramas such as 7 first kisses didn't get picked up although there is not issue with serving via the api. This because the name found on wikipedia is not the same as the "title" name on imdb's website.

For these how about we eliminate the restriction on the title.

In [12]:
temp = wikiKdramasDf[wikiKdramasDf['title'] == "7 First Kisses"]
for idx, d in temp.iterrows():
    title = d['title']
    year = d['year']
    try:
        dramas = ia.search_movie_advanced(title)
        print(dramas)
        for dr in dramas:
            try:
                print(dr['title'], title)
                if (dr['title'] == title):
                    print("here")
                    print(title, dr['year'], year)
                    if (dr['year'] == year):
                        print("passed test", dr.movieID)
            except:
                print('error')
    except:
        print("too may values")
#ia.search_movie_advanced("7 First Kisses")[0]['year']

[<Movie id:6353134[http] title:_"Seven First Kisses" (2016)_>]
Seven First Kisses 7 First Kisses


In [13]:
noMovieId = wikiKdramasDf.loc[(wikiKdramasDf['movie_id']).isnull()]

for idx, d in noMovieId.iterrows():
    try:
        title = d['title']
        print(title)
        year = d['year']
        drama_names = ia.search_movie_advanced(title)
        found = []
        for dr in drama_names:
            try:
                if (dr['year'] == year):
                    found.append(dr)
                    print(title, 'found')
            except:
                print(title, 'year not found')
        print(found)
        if len(found) > 1:
            if len(found) > 8:
                wikiKdramasDf.loc[wikiKdramasDf['title'] == title, 'movie_id'] = found[0].movieID
            else:
                wikiKdramasDf.loc[wikiKdramasDf['title'] == title, 'movie_id'] = ','.join([x.movieID for x in found])
            print('multiple id')
        elif len(found) == 1:
            wikiKdramasDf.loc[wikiKdramasDf['title'] == title, 'movie_id'] = found[0].movieID
            print('single id')
        else:
            print('no id')
    except:
        print('error')


12 Signs of Love
[]
no id
4 Legendary Witches
[]
no id
5th Republic
[]
no id
7 First Kisses
7 First Kisses found
[<Movie id:6353134[http] title:_"Seven First Kisses" (2016)_>]
single id
90 Days, Time to Love
[]
no id
A Beautiful Mind
A Beautiful Mind found
A Beautiful Mind found
[<Movie id:12597746[http] title:_" Based on a True Story" (2016)_>, <Movie id:16252662[http] title:_" Timesuck with Dan Cummins" (2016)_>]
multiple id
A Bird That Doesn't Sing
[]
no id
A Girl Who Sees Smells
[]
no id
A Happy  Woman
A Happy  Woman found
[<Movie id:5621162[http] title:_A Happy Woman (2007)_>]
single id
A Hundred Year Legacy
[]
no id
A Little Love Never Hurts
[]
no id
A-Teen 2
A-Teen 2 found
A-Teen 2 found
[<Movie id:11955552[http] title:_"A-Teen" The Story Of A-TEEN's Two Hanas + Easter Eggs + Spoilers (2019)_>, <Movie id:11959084[http] title:_" True Crime Daily" (2019)_>]
multiple id
Ad Genius Lee Tae-baek
Ad Genius Lee Tae-baek found
Ad Genius Lee Tae-baek found
[<Movie id:3541250[http] title:_

2022-10-31 23:50:09,539 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


error
Be Strong, Geum-soon!
Be Strong, Geum-soon! found
[<Movie id:9165876[http] title:_Be Strong, Geum-Soon! (2005)_>]
single id
Beating Again
[]
no id
Because This Is My First Life
Because This Is My First Life found
Because This Is My First Life found
[<Movie id:7278588[http] title:_Because This Is My First Life (2017)_>, <Movie id:7611220[http] title:_Because This Is My First Life Because This Is My First Life (2017)_>]
multiple id
Because This Is My First Twenty
[]
no id
Becky's Back
[]
no id
Becoming a Billionaire
Becoming a Billionaire found
[<Movie id:15360196[http] title:_"Happy Together" Co-stars in Becoming a Billionaire: Lee Bo-young, Ji Hyun-woo, Lee Si-young, Kim Byung-man, Ryu Dam (2010)_>]
single id
Bel Ami
[]
no id
Beyond Evil


2022-10-31 23:50:18,913 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


error
Big
[]
no id
Big Issue
Big Issue found
Big Issue year not found
Big Issue found
[<Movie id:9772798[http] title:_Big Issue (2019)_>, <Movie id:10101808[http] title:_"The Phlaash" Big Issue (2019)_>]
multiple id
Big Thing
Big Thing found
Big Thing found
[<Movie id:3406906[http] title:_Next Big Thing (2010) (V)_>, <Movie id:7513036[http] title:_Audi the Next Big Thing (2010)_>]
multiple id
Billie Jean, Look at Me
[]
no id
Birth of a Beauty
Birth of a Beauty found
[<Movie id:4269776[http] title:_"Birth of the Beauty" (2014)_>]
single id
Birthday Letter
Birthday Letter found
[<Movie id:15831086[http] title:_" Full Body Chills" (2019)_>]
single id
Bitter Sweet Life
[]
no id
Bizarre Bunch
[]
no id
Black
Black found
[<Movie id:7441658[http] title:_"Black Clover" (2017)_>]
single id
Black Dog: Being A Teacher
[]
no id
Black Knight
Black Knight year not found
Black Knight year not found
Black Knight year not found
Black Knight year not found
Black Knight year not found
[]
no id
Black Knigh

...

error
The Producers
The Producers found
The Producers year not found
The Producers found
The Producers found
The Producers year not found
[<Movie id:4612922[http] title:_Peurodyusa (2015)_>, <Movie id:4462166[http] title:_The Drew: No Excuse, Just Produce (2015)_>, <Movie id:2544624[http] title:_The Hit Producer (2015)_>]
multiple id
The Rebel
The Rebel found
The Rebel found
[<Movie id:4986134[http] title:_Rebel in the Rye (2017)_>, <Movie id:6256694[http] title:_The Rebel Hong Gil Dong (2017)_>]
multiple id
The Return of Hwang Geum-bok
The Return of Hwang Geum-bok found
[<Movie id:5688566[http] title:_"The Return of Hwang Geum-Bok" (2015)_>]
single id
The Rich Son
[]
no id
The Road Home
The Road Home year not found
[]
no id
The Road: The Tragedy of One
The Road: The Tragedy of One found
[<Movie id:14687098[http] title:_"The Road: Tragedy of One" (2021)_>]
single id
The Scandal
The Scandal found
The Scandal year not found
[<Movie id:3144390[http] title:_The Secret Scandal (2013)_>]
sin

2022-10-31 23:58:51,953 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


error
This Is My Love
[]
no id
Three Bold Siblings
[]
no id
Queen of the Ring
Queen of the Ring year not found
Queen of the Ring found
Queen of the Ring found
Queen of the Ring found
Queen of the Ring found
[<Movie id:8023204[http] title:_WSU King and Queen of the Ring (2017) (V)_>, <Movie id:17594944[http] title:_" WrestleTalk Podcast" (2017)_>, <Movie id:17594944[http] title:_" WrestleTalk Podcast" (2017)_>, <Movie id:17594944[http] title:_" WrestleTalk Podcast" (2017)_>]
multiple id
Three Days
Three Days found
Three Days year not found
[<Movie id:2172934[http] title:_3 Days to Kill (2014)_>]
single id
Three Friends
[]
no id
Three Sisters
Three Sisters year not found
Three Sisters year not found
[]
no id
Thrice Married Woman
[]
no id
Through the Darkness
Through the Darkness year not found
Through the Darkness found
Through the Darkness found
Through the Darkness found
[<Movie id:13655044[http] title:_Break Through the Darkness (2021)_>, <Movie id:15908898[http] title:_Go Through the

2022-10-31 23:59:12,792 ERROR [imdbpy.parser.http.domparser] /Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py:441: DOMHTMLSearchMovieAdvancedParser: caught exception postprocessing data
Traceback (most recent call last):
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/utils.py", line 438, in parse
    data = self.postprocess_data(data)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 232, in postprocess_data
    secondary = _parse_secondary_info(secondary_info)
  File "/Users/divineechidume/opt/anaconda3/lib/python3.7/site-packages/imdb/parser/http/searchMovieAdvancedParser.py", line 52, in _parse_secondary_info
    if match.group(2):
AttributeError: 'NoneType' object has no attribute 'group'


error
Trolley
Trolley year not found
Trolley found
Trolley year not found
[<Movie id:19823068[http] title:_Trolley Problem, Inc. (2022) (VG)_>]
single id
Twenty-Five Twenty-One
Twenty-Five Twenty-One found
[<Movie id:17513352[http] title:_Twenty Five Twenty One (2022)_>]
single id
Two Outs in the Ninth Inning
[]
no id
Two Weeks
Two Weeks found
[<Movie id:2964526[http] title:_2 Weeks (2013)_>]
single id
Two Wives
[]
no id
Ugly Miss Young-ae
[]
no id
Uncle
[]
no id
Under the Queen's Umbrella
Under the Queen's Umbrella found
[<Movie id:22265394[http] title:_The Queen's Umbrella (2022)_>]
single id
Undercover
Undercover found
Undercover found
Undercover found
[<Movie id:12940504[http] title:_My Name (2021)_>, <Movie id:13634530[http] title:_"Undercover" (2021)_>, <Movie id:8415986[http] title:_Undercover (2021)_>]
multiple id
Understanding of Love
[]
no id
Unlock the Boss
[]
no id
Unstoppable Marriage
Unstoppable Marriage found
Unstoppable Marriage found
[<Movie id:2315142[http] title:_"Un

In [17]:
wikiKdramasDf[~wikiKdramasDf['movie_id'].isnull()].shape

(1105, 4)

In [15]:
wikiKdramasDf.to_csv("minedKdramas.csv", index = False)

In [16]:
wikiKdramasDf

,title,year,endYear,movie_id
0,100 Days My Prince,2018,2018,8199972
1,12 Signs of Love,2012,2012,NaN
2,12 Years Promise,2014,2014,5476252
3,18 Again,2020,2020,12846096
4,365: Repeat the Year,2020,2020,12015466
...,...,...,...,...
1569,Your Scene,2007,2007,NaN
1570,Youth,2021,2021,14166666
1571,Youth of May,2021,2021,14166666
1572,Yumi's Cells,2021,2022,14596630


In [135]:
ia.get_movie_infoset()

['airing',
 'akas',
 'alternate versions',
 'awards',
 'connections',
 'crazy credits',
 'critic reviews',
 'episodes',
 'external reviews',
 'external sites',
 'faqs',
 'full credits',
 'goofs',
 'keywords',
 'list',
 'locations',
 'main',
 'misc sites',
 'news',
 'official sites',
 'parents guide',
 'photo sites',
 'plot',
 'quotes',
 'recommendations',
 'release dates',
 'release info',
 'reviews',
 'sound clips',
 'soundtrack',
 'synopsis',
 'taglines',
 'technical',
 'trivia',
 'tv schedule',
 'video clips',
 'vote details']

In [324]:
wikiKdramasDf[wikiKdramasDf['movie_id'].isnull()]

KeyError: 'movie_id'